In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/sign-language-mnist/sign_mnist_test.csv
/kaggle/input/sign-language-mnist/amer_sign2.png
/kaggle/input/sign-language-mnist/amer_sign3.png
/kaggle/input/sign-language-mnist/sign_mnist_train.csv
/kaggle/input/sign-language-mnist/american_sign_language.PNG
/kaggle/input/sign-language-mnist/sign_mnist_test/sign_mnist_test.csv
/kaggle/input/sign-language-mnist/sign_mnist_train/sign_mnist_train.csv


In [18]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim import lr_scheduler
import torchvision
from torchvision import datasets,models,transforms
import matplotlib.pyplot as plt
import time
import copy
from PIL import Image
from tempfile import TemporaryDirectory
from torch.utils.data import Dataset, DataLoader
from torchvision.io import read_image

In [16]:
train_data = pd.read_csv('/kaggle/input/sign-language-mnist/sign_mnist_train/sign_mnist_train.csv')
test_data = pd.read_csv('/kaggle/input/sign-language-mnist/sign_mnist_test/sign_mnist_test.csv')
# Extract labels and pixel data
train_labels = train_data['label'].values
test_labels = test_data['label'].values
train_pixels = train_data.iloc[:, 1:].values.astype('float32')
test_pixels = test_data.iloc[:, 1:].values.astype('float32')

# Normalize pixel values to range [0,1]
train_pixels /= 255.0
test_pixels /= 255.0

# Reshape the images (each image is 28x28, grayscale)
train_pixels = train_pixels.reshape(-1, 1, 28, 28)
test_pixels = test_pixels.reshape(-1,1,28,28)

# Convert NumPy arrays to PyTorch tensors 
train_pixels_tensor = torch.tensor(train_pixels)
train_labels_tensor = torch.tensor(train_labels)

test_pixels_tensor = torch.tensor(test_pixels)
test_labels_tensor = torch.tensor(test_labels)

In [17]:
# Defining a custom dataset class
class SignLanguageDataset(Dataset):
    def __init__(self, images, labels):
        self.images = images
        self.labels = labels
    
    def __len__(self):
        return len(self.labels)
    
    def __getitem__(self, idx):
        image = self.images[idx]
        label = self.labels[idx]
        return image, label
    
# Create datasets and data loaders
train_dataset = SignLanguageDataset(train_pixels_tensor, train_labels_tensor)
test_dataset = SignLanguageDataset(test_pixels_tensor, test_labels_tensor)

batch_size = 64
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size = batch_size, shuffle=False)

In [27]:
# Defining the Neural Network Model
class SignLanguageNet(nn.Module):
    def __init__(self):
        super(SignLanguageNet, self).__init__()
        self.conv1 = nn.Conv2d(1,16,kernel_size=3,stride=1,padding=1)
        self.conv2 = nn.Conv2d(16,32, kernel_size=3,stride=1,padding=1)
        self.fc1 = nn.Linear(32 * 7 * 7, 128)
        self.fc2 = nn.Linear(128,25)
        
    def forward(self,x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, kernel_size=2, stride=2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, kernel_size=2, stride=2)
        x = x.view(-1, 32 * 7 * 7)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Create an instance of the model
model = SignLanguageNet()

In [29]:
# Training the model

# define the loss function and the optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)

num_epochs = 25
device = torch.device("cuda" if torch.cuda.is_available() else 'cpu')
model.to(device)

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs,labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item() * images.size(0)
    epoch_loss = running_loss / len(train_loader.dataset)
    print(f'Epoch {epoch + 1}/{num_epochs}, Loss: {epoch_loss:.4f}')
    
print('Finished Training')

Epoch 1/25, Loss: 0.4485
Epoch 2/25, Loss: 0.3570
Epoch 3/25, Loss: 0.2896
Epoch 4/25, Loss: 0.2149
Epoch 5/25, Loss: 0.1701
Epoch 6/25, Loss: 0.1364
Epoch 7/25, Loss: 0.1105
Epoch 8/25, Loss: 0.0818
Epoch 9/25, Loss: 0.0705
Epoch 10/25, Loss: 0.0568
Epoch 11/25, Loss: 0.0390
Epoch 12/25, Loss: 0.0417
Epoch 13/25, Loss: 0.0235
Epoch 14/25, Loss: 0.0466
Epoch 15/25, Loss: 0.0198
Epoch 16/25, Loss: 0.0128
Epoch 17/25, Loss: 0.0109
Epoch 18/25, Loss: 0.0104
Epoch 19/25, Loss: 0.0082
Epoch 20/25, Loss: 0.0081
Epoch 21/25, Loss: 0.0867
Epoch 22/25, Loss: 0.0168
Epoch 23/25, Loss: 0.0077
Epoch 24/25, Loss: 0.0062
Epoch 25/25, Loss: 0.0768
Finished Training


In [32]:
# Model Evaluation
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        
accuracy = correct / total
print(f'Accuracy on test set: {accuracy:.2%}')

Accuracy on test set: 86.56%
